In [1]:
import pandas as pd
import numpy as np

In [35]:
df=pd.read_csv('u.data',sep='\t')

In [36]:
df.columns=['user_id', 'item_id', 'rating', 'time']

In [37]:
df.head()

,user_id,item_id,rating,time
0,186,302,3,891717742
1,22,377,1,878887116
2,244,51,2,880606923
3,166,346,1,886397596
4,298,474,4,884182806


In [38]:
from datetime import datetime

In [39]:
df['time']=df['time'].apply(lambda x:datetime.fromtimestamp(x/1))

In [40]:
df.head()

,user_id,item_id,rating,time
0,186,302,3,1998-04-04 14:22:22
1,22,377,1,1997-11-07 01:18:36
2,244,51,2,1997-11-26 23:02:03
3,166,346,1,1998-02-01 23:33:16
4,298,474,4,1998-01-07 08:20:06


In [49]:
rate=pd.pivot_table(df,index=['user_id'],columns=['item_id'],values=['rating'])

In [50]:
rate.columns=rate.columns.droplevel()

In [51]:
rate.head()

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
1-rate.isnull().sum().sum()/rate.shape[1]/rate.shape[0]

0.06304606317530892

In [58]:
rate=rate.fillna(0)

In [59]:
rate.head()

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
from sklearn.model_selection import train_test_split

In [62]:
rate_train,rate_test=train_test_split(rate,test_size=0.1)

In [63]:
from sklearn.preprocessing import normalize

In [240]:
def similarity(df,kind='user'):
    if kind=='user':  
        y=df.index
    elif kind=='item': 
        y=df.columns
        df=df.T
    df=normalize(df,axis=1)
    x=pd.DataFrame(df.dot(df.T),index=y,columns=y)
    return(x)

In [241]:
item=similarity(rate_train,'item')
user=similarity(rate_train,'user')

In [237]:
def recommend(user_df,rate_df):
    user_df=user_df.values
    np.fill_diagonal(user_df,0)
    user_df=user_df/user_df.sum(axis=0)
    x=user_df.T.dot(rate_df)
    x=pd.DataFrame(x,index=rate_df.index,columns=rate_df.columns)
    return(x)

In [243]:
recommend(user,rate_train)

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
762,1.833138,0.392272,0.343958,0.806719,0.264180,0.127541,1.680143,0.901894,1.393031,0.418952,...,0.002340,0.002810,0.004560,0.003040,0.006577,0.001364,0.004093,0.002728,0.0,0.004668
599,2.467098,0.430238,0.436373,0.725464,0.339314,0.085854,1.890438,0.838819,1.489278,0.319367,...,0.003093,0.001324,0.003605,0.002403,0.005966,0.000911,0.002733,0.001822,0.0,0.003560
175,2.426527,0.738782,0.408620,1.295799,0.459787,0.110331,2.119857,1.453131,1.631673,0.438811,...,0.005447,0.009164,0.005941,0.003961,0.005799,0.000000,0.000000,0.000000,0.0,0.006320
668,1.688858,0.398199,0.305582,0.701272,0.248570,0.092997,1.484596,0.785324,1.114168,0.317661,...,0.002613,0.002723,0.004227,0.002818,0.003502,0.002521,0.007564,0.005043,0.0,0.002212
400,1.315235,0.237619,0.210801,0.428579,0.177553,0.067274,1.144376,0.511460,0.900776,0.228515,...,0.001677,0.000358,0.003782,0.002522,0.002479,0.003696,0.011089,0.007393,0.0,0.000427
906,2.267254,0.403755,0.420410,0.775432,0.296813,0.143700,2.122245,0.880973,1.775886,0.486420,...,0.002653,0.003453,0.005210,0.003473,0.006563,0.000805,0.002415,0.001610,0.0,0.003584
757,2.470616,0.809913,0.473731,1.272635,0.468186,0.092970,2.164586,1.309976,1.425733,0.414269,...,0.002884,0.005626,0.004955,0.003303,0.004678,0.000565,0.001696,0.001131,0.0,0.007117
28,2.256773,0.700656,0.394400,1.174889,0.485035,0.098037,2.134275,1.250534,1.396147,0.402900,...,0.003217,0.006331,0.003701,0.002467,0.005108,0.000618,0.001854,0.001236,0.0,0.005367
583,2.085531,0.567386,0.387864,1.115981,0.382116,0.142410,2.123856,1.216294,1.551533,0.468952,...,0.002780,0.008537,0.001767,0.001178,0.006070,0.000790,0.002370,0.001580,0.0,0.006924


In [254]:
def get_topk(item_df,k):
    x=item_df.sort_values(ascending=False)[1:k+1].index.values
    return(pd.Series(x,index=range(1,k+1)))

In [255]:
item.apply(get_topk,k=3,axis=1)

,1,2,3
item_id,,,
1,50,121,181
2,233,161,385
3,410,42,763
4,56,174,204
5,218,219,53
6,1370,766,1129
7,100,50,117
8,204,423,69
9,100,276,7


In [302]:
def predict_score(user_df,rate_df,user,item):
    x=(rate_df[item]!=0).index.values  #user who rate item
    user_df=user_df.loc[user,x]/user_df.loc[user,x].sum()
    rate_df=rate_df.loc[x,item]
    return(user_df.dot(rate_df))

In [318]:
predict_score(user,rate_train,762,2)

0.3922719185011031